In [ ]:
!pip install --upgrade nibabel

Get data


In [ ]:
!wget "https://putpoznanpl-my.sharepoint.com/:u:/g/personal/dominik_pieczynski_put_poznan_pl/EWIZ_xm8wXpMjQDgF2VQ1csB4QuHPKoj5vDpj6CQi9p-AA?e=yQr6fn&download=1" -O public.zip
!unzip -q public.zip
!rm public.zip

Create valid dirs

In [3]:
!mkdir /content/FirstDataset/valid
!mkdir /content/SecondDataset/valid

Checking the number of scans in dirs 

In [4]:
import os
import shutil

print("FirstDataset")
filepath1 = '/content/FirstDataset/train'
filenames1 = os.listdir(filepath1)
filenames1 = sorted(filenames1)
print('train', len(filenames1))
print('test', len(os.listdir('/content/FirstDataset/test')))
print(int(len(filenames1)/5)) #114

print("SecondDataset")
filepath2 = '/content/SecondDataset/train'
filenames2 = os.listdir(filepath2)
filenames2 = sorted(filenames2)
print('train', len(filenames2))
print('test', len(os.listdir('/content/SecondDataset/test')))
print(int(len(filenames2)/5)) # 20

FirstDataset
train 574
test 72
114
SecondDataset
train 100
test 25
20


Transfer of some data from train dir to valid dir

In [5]:
import os
import shutil

# FirstDataset
filepath1 = '/content/FirstDataset/train'
filenames1 = os.listdir(filepath1)
filenames1 = sorted(filenames1)
# print(len(filenames1))
valid_filenames1 = filenames1[:int(len(filenames1)/5)]
# print(valid_filenames1)
for file_name in valid_filenames1:
  shutil.move(f"/content/FirstDataset/train/{file_name}", f"/content/FirstDataset/valid/{file_name}")
print("train len:", len(os.listdir('/content/FirstDataset/train')))
print("valid len:", len(os.listdir('/content/FirstDataset/valid')))
print('test len', len(os.listdir('/content/FirstDataset/test')))
print('finish 1')

# SecondDataset
filepath2 = '/content/SecondDataset/train'
filenames2 = os.listdir(filepath2)
filenames2 = sorted(filenames2)
# print(len(filenames2))
valid_filenames2 = filenames2[:int(len(filenames2)/5)]
# print(valid_filenames2)
for file_name in valid_filenames2:
  shutil.move(f"/content/SecondDataset/train/{file_name}", f"/content/SecondDataset/valid/{file_name}")
print("train len:", len(os.listdir('/content/SecondDataset/train')))
print("valid len:", len(os.listdir('/content/SecondDataset/valid')))
print('test len', len(os.listdir('/content/SecondDataset/test')))
print('finish 2')

train len: 460
valid len: 114
test len 72
finish 1
train len: 80
valid len: 20
test len 25
finish 2


###Eksport to png

Template methods

In [6]:
import numpy as np
import nibabel as nib

from typing import Tuple, List
from pathlib import Path


def load_raw_volume(path: Path) -> Tuple[np.ndarray, np.ndarray]:
  data: nib.Nifti1Image = nib.load(str(path))
  data = nib.as_closest_canonical(data)
  raw_data = data.get_fdata(caching='unchanged', dtype=np.float32)
  return raw_data, data.affine


def load_labels_volume(path: Path) -> np.ndarray:
  return load_raw_volume(path)[0].astype(np.uint8)


def save_labels(data: np.ndarray, affine: np.ndarray, path: Path):
  nib.save(nib.Nifti1Image(data, affine), str(path))


def show_slices(slices: List[np.ndarray]):
   fig, axes = plt.subplots(1, len(slices))
   for i, data_slice in enumerate(slices):
       axes[i].imshow(data_slice.T, cmap="gray", origin="lower")

Create dirs for images

In [47]:
!rm -rf FirstDataset_images


!mkdir /content/FirstDataset_images

!mkdir /content/FirstDataset_images/train
!mkdir /content/FirstDataset_images/train/images
!mkdir /content/FirstDataset_images/train/masks

!mkdir /content/FirstDataset_images/valid
!mkdir /content/FirstDataset_images/valid/images
!mkdir /content/FirstDataset_images/valid/masks

Export to png

In [ ]:
import matplotlib.pyplot as plt
import cv2


first_dataset_path = Path('/content/FirstDataset/train')
first_image_dataset_path = Path('/content/FirstDataset_images/train/images')
first_mask_dataset_path = Path('/content/FirstDataset_images/train/masks')


for scan_path in first_dataset_path.iterdir():
  print(scan_path)
  raw_volume, affine = load_raw_volume(scan_path)
  mask_volume = load_labels_volume(scan_path)
  if scan_path.name.endswith('mask.nii.gz'):
    for idx in range(mask_volume.shape[0]):
      path = f'/content/FirstDataset_images/train/masks/{scan_path.name}_{idx}_.png'
      plt.imsave(path, mask_volume[idx])
  else:
    for idx in range(raw_volume.shape[0]):
      path = f'/content/FirstDataset_images/train/images/{scan_path.name}_{idx}_.png'
      plt.imsave(path, raw_volume[idx])


print("TRAIN DATASET FINISHED")

first_dataset_path = Path('/content/FirstDataset/valid')
first_image_dataset_path_valid = Path('/content/FirstDataset_images/valid/images')
first_mask_dataset_path_valid = Path('/content/FirstDataset_images/valid/masks')

for scan_path in first_dataset_path.iterdir():
  print(scan_path)
  raw_volume, affine = load_raw_volume(scan_path)
  mask_volume = load_labels_volume(scan_path)
  if scan_path.name.endswith('mask.nii.gz'):
    for idx in range(mask_volume.shape[0]):
      path = f'/content/FirstDataset_images/valid/masks/{scan_path.name}_{idx}_.png'
      plt.imsave(path, mask_volume[idx])
  else:
    for idx in range(raw_volume.shape[0]):
      path = f'/content/FirstDataset_images/valid/images/{scan_path.name}_{idx}_.png'
      plt.imsave(path, raw_volume[idx])


print("VALID DATASET FINISHED")
